In [87]:
import pandas as pd

# Read CSV file into a DataFrame
df = pd.read_csv("DailyPrices.csv")

# Calculate arithmetic returns for all prices
returns = return_calculate(df, method="DISCRETE", dateColumn="Date")

# Print the first few rows of the returns DataFrame
print(returns.head())


             Date       SPY      AAPL      MSFT      AMZN      TSLA     GOOGL  \
1  2/15/2022 0:00  0.016127  0.023152  0.018542  0.008658  0.053291  0.007987   
2  2/16/2022 0:00  0.001121 -0.001389 -0.001167  0.010159  0.001041  0.008268   
3  2/17/2022 0:00 -0.021361 -0.021269 -0.029282 -0.021809 -0.050943 -0.037746   
4  2/18/2022 0:00 -0.006475 -0.009356 -0.009631 -0.013262 -0.022103 -0.016116   
5  2/22/2022 0:00 -0.010732 -0.017812 -0.000729 -0.015753 -0.041366 -0.004521   

       GOOG      META      NVDA  ...       PNC      MDLZ        MO       ADI  \
1  0.008319  0.015158  0.091812  ...  0.012807 -0.004082  0.004592  0.052344   
2  0.007784 -0.020181  0.000604  ...  0.006757 -0.002429  0.005763  0.038879   
3 -0.037669 -0.040778 -0.075591  ... -0.034949  0.005326  0.015017 -0.046988   
4 -0.013914 -0.007462 -0.035296  ... -0.000646 -0.000908  0.007203 -0.000436   
5 -0.008163 -0.019790 -0.010659  ...  0.009494  0.007121 -0.008891  0.003243   

       GILD       LMT       SYK 

/tmp/ipykernel_1663/4141197605.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


In [23]:
import numpy as np

# Calculate the mean of the META returns series
meta_mean = np.mean(modeling_meta_returns)

# Subtract the mean from the META returns series
modeling_meta_returns_centered = modeling_meta_returns - meta_mean

print(modeling_meta_returns_centered)

# Print the mean of the centered META returns series
print(f"Centered META mean: {np.mean(modeling_meta_returns_centered)}")


         META
1    0.017826
2   -0.017513
3   -0.038110
4   -0.004795
5   -0.017123
..        ...
184  0.067983
185  0.000083
186  0.054497
187  0.105161
188  0.012948

[188 rows x 1 columns]
Centered META mean: META    9.227253e-19
dtype: float64


/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [77]:
#Using a normal distribution.
import pandas as pd
import numpy as np
from scipy.stats import norm


sd = np.std(modeling_meta_returns_centered)

# Calculate the VaR at the 5% and 1% confidence levels
VaR_05 = -norm.ppf(0.05, 0, sd)
VaR_01 = -norm.ppf(0.01, 0, sd)

print(VaR_05)
#print(VaR_01)
# Print the results
#print(f"VaR using a normal distribution (5% confidence level): {VaR_05:.2f}")


[0.06534605]


In [85]:

def calculate_var(data, mean=0, alpha=0.05):
  return mean - np.quantile(data, alpha)

def calculate_exponential_weights(lags, l):
  weights = []
  for i in range(1, lags + 1):
    weight = (1 - l) * l ** (i - 1)
    weights.append(weight)
  weights = np.array(weights)
  weights = np.flip(weights)
  normalized_weights = weights / weights.sum()
  return normalized_weights

def calculate_ewcov(data, l):
  w = calculate_exponential_weights(data.shape[1], l)
  error_matrix = data - data.mean(axis=1)
  ewcov = error_matrix @ np.diag(w) @ error_matrix.T
  return ewcov


ew_cov = calculate_ewcov(np.matrix(modeling_meta_returns_centered).T, 0.94)
ew_variance = ew_cov[0, 0]
sigma = np.sqrt(ew_variance)
simulation_ew = np.random.normal(0, sigma, 10000)
var_ew = calculate_var(simulation_ew)
print(var_ew)

0.09669898426573666


In [75]:
#Using a MLE fitted T distribution.

from scipy.stats import t

# Fit a T distribution to the daily returns
params = t.fit(modeling_meta_returns_centered)

# Set the confidence level
confidence_level = 0.95

# Calculate the VaR at the given confidence level
var = t.ppf(1 - confidence_level, *params[:-2], loc=params[-2], scale=params[-1])

print("VaR at 95% confidence level using an MLE model:",var)


VaR at 95% confidence level using an MLE model: -0.059015163290606514


In [42]:
#Using a fitted AR(1) model.
import pandas as pd
import numpy as np
import statsmodels.api as sm


# Fit an AR(1) model to the log returns
model = sm.tsa.ARIMA(modeling_meta_returns_centered, order=(1,0,0))
results = model.fit()

# Calculate the standard deviation of the residuals
sigma = np.sqrt(results.resid.var())

# Set the confidence level and calculate the z-score
alpha = 0.05
z_alpha = norm.ppf(1-alpha)

# Calculate the VaR
VaR = - z_alpha * sigma 

print("VaR at 95% confidence level using an AR(1) model: ", VaR)



VaR at 95% confidence level using an AR(1) model:  -0.06551931823069516


In [70]:
#Using a Historic Simulation.

# Set the current value of the asset
S = 100

# Sort the returns in descending order
sorted_returns = sorted(modeling_meta_returns_centered['META'].dropna(), reverse=True)

alpha = 0.05

# Calculate the index at which the VaR cutoff occurs
n = len(sorted_returns)
cutoff_index = int(np.floor(n * alpha))

# Calculate the VaR

VaR = - sorted_returns[cutoff_index]

print("VaR at 95% confidence level using historic simulation: ", VaR)

VaR at 95% confidence level using historic simulation:  -0.05684504307257836
